### Two transmon optimisation with cosine signal
Same as two_transmons_4LOs.ipynb, but the signal is created by a superposition of cosines. This can be used to optimise a large superposition of frequency peaks.

In [1]:
import argparse
import copy
import os
import sys

import numpy as np
import tensorflow as tf

import c3
import c3.libraries.algorithms as algorithms
import c3.libraries.fidelities as fidelities
import c3.utils.qt_utils as qt_utils
from c3.signal.pulse import EnvelopeDrag
from c3.utils.tf_utils import tf_project_to_comp
import c3.utils.tf_utils as tf_utils
import four_level_transmons.custom_gates as custom_gates
from c3.experiment import Experiment as Exp
from c3.model import Model as Mdl
from c3.optimizers.optimalcontrol import OptimalControl
from c3.parametermap import ParameterMap as PMap
from four_level_transmons.DataOutput import DataOutput
from four_level_transmons.custom_envelopes import *
from four_level_transmons.plotting import *
from four_level_transmons.utilities import *
from four_level_transmons.blackbox import generateSignalFromConfig
from four_level_transmons.notebook_utils import *
from four_level_transmons.input import *
tf.config.run_functions_eagerly(True)
np.set_printoptions(linewidth=300)

2023-11-02 14:05:09.799497: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-02 14:05:09.799557: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Parse parameters from the openstack scripts
FREQS = None
FREQ_INDEX = None
INPUT_FILE = None
if len(sys.argv[1:]) > 3 and "ipykernel_launcher" not in sys.argv[0]:
    parser = argparse.ArgumentParser()
    parser.add_argument("--output", help="Output directory")
    parser.add_argument("--config", help="File with previous configuration")
    parser.add_argument("--freqs1", help="New number of frequencies on qubit1")
    parser.add_argument("--freqs2", help="New number of frequencies on qubit2")
    args = parser.parse_args()
    output_dir = args.output
    INPUT_FILE = args.config
    FREQS = (int(args.freqs1), int(args.freqs2))
    print("Output directory: ", output_dir)
    print("Number of frequencies: ", FREQS)
elif len(sys.argv[1:]) > 2 and "ipykernel_launcher" not in sys.argv[0]:
    parser = argparse.ArgumentParser()
    parser.add_argument("--output", help="Output directory")
    parser.add_argument("--config", help="File with previous configuration")
    parser.add_argument("--freqindex", help="Index of the  frequency to be removed")
    args = parser.parse_args()
    output_dir = args.output
    INPUT_FILE = args.config
    FREQ_INDEX = int(args.freqindex)
    print("Output directory: ", output_dir)
    print("Removed frequency: ", FREQS)
elif len(sys.argv[1:]) > 0 and "ipykernel_launcher" not in sys.argv[0]:
    parser = argparse.ArgumentParser()
    parser.add_argument("--output", help="Output directory")
    args = parser.parse_args()
    output_dir = args.output
    print("Output directory: ", output_dir)
else:
    print("=========== WARNING: no output directory specified ============")
    output_dir = "./output"
output = DataOutput(output_dir, file_suffix='before')

=========== WARNING: no output directory specified ============


### Settings

In [3]:
# General settings
numPWCPieces = 60
usePWC = False
useDRAG = False
t_final = 1000e-9
sim_res = 20e9
awg_res = numPWCPieces / t_final if usePWC else 10e9 #1000e9
isDressed = True
useFR = False
OPTIMISE_TFINAL = False
OPTIMISE_FREQUENCIES = False
OPTIMISABLE_DRIVES = [0, 1]
USE_COMPLEX_GAUSSIANS = True
USE_COMPLEX_AMPLITUDE = True

# Optimisation algorithm and parameters
ALGORITHM_LBFGS = 0
ALGORITHM_LBFGS_GRAD_FREE = 1
ALGORITHM_CMAES = 2
ALGORITHM_GCMAES = 3
selected_algorithms = [
    #(ALGORITHM_LBFGS, {"maxfun": 50}),
    #(ALGORITHM_CMAES, {}),
    (ALGORITHM_LBFGS, {"maxfun": 1500, "ftol": 1e-6})
]

# Input file, if necessary
#if INPUT_FILE is None:
#    INPUT_FILE = "./optimised_params/two transmons/fourier basis/1000ns/CZ neu/10.json"

# Ideal gate
#IDEAL_GATE, IDEAL_GATE_NAME = np.eye(16), "unity"
#IDEAL_GATE, IDEAL_GATE_NAME = custom_gates.GATE_CNOT_t1q2_t2q2, "cnot_t1q2_t2q2"
#IDEAL_GATE, IDEAL_GATE_NAME = custom_gates.GATE_iCNOT_t1q2_t2q2, "icnot_t1q2_t2q2"
#IDEAL_GATE, IDEAL_GATE_NAME = custom_gates.GATE_iSWAP_t1q2_t2q2, "iswap_t1q2_t2q2"
#IDEAL_GATE, IDEAL_GATE_NAME = custom_gates.GATE_SQRTiSWAP_t1q2_t2q2, "sqrtiswap_t1q2_t2q2"
IDEAL_GATE, IDEAL_GATE_NAME = custom_gates.GATE_CZ_t1q2_t2q2, "cz_t1_t2"
#IDEAL_GATE, IDEAL_GATE_NAME = custom_gates.GATE_4QUBIT_1110_1111, "cccnot_1110"
#IDEAL_GATE, IDEAL_GATE_NAME = custom_gates.GATE_4QUBIT_CCCZ, "cccz"

#NUM_PEAKS = (200, 200)
#INCLUDED_INDICES = ([1,2,3,4,5,6,7,8,9,10,11,12,13,14], None)

# Limit the number of frequencies, select individual ones, or specify a frequency range
if FREQS is not None:
    NUM_PEAKS = FREQS
    #NUM_PEAKS = (FREQS[0], 0)
else:
    NUM_PEAKS = (200, 200)
#NUM_PEAKS = (-1, -1)

if FREQ_INDEX is not None:
    print("Removing frequency ", FREQ_INDEX) ###REMOVE
    frequencies = list(range(NUM_PEAKS[0]))
    frequencies.remove(FREQ_INDEX)
    INCLUDED_INDICES = (frequencies, frequencies)
else:
    INCLUDED_INDICES = (None, None)
#INCLUDED_INDICES = (np.array([1,2,3,4,5,6,7,8,9,10,11,12]), None)

SPECTRAL_RANGE = (4.5e9, 6.3e9)
#SPECTRAL_RANGE = (-1, -1)
FREQUENCIES_IN_RANGE = 200

### Initialise the model: transmons, coupling, and drives

In [7]:
qubit_levels = [5, 5]
qubit_frequencies = [6e9, 5.6e9] #[5e9, 4.5e9]
anharmonicities = [-240e6, -220e6] #[-300e6, -250e6]
t1s = [25e-6, 25e-6]
t2stars = [35e-6, 35e-6]
qubit_temps = [50e-3, 50e-3]
couplingStrength = 10e6
print("qubits frequencies: ", qubit_frequencies, "anharmonicities: ", anharmonicities, "coupling: ", couplingStrength)

qubits = createQubits(qubit_levels, qubit_frequencies, anharmonicities, t1s, t2stars, qubit_temps)
coupling = createChainCouplings([couplingStrength], qubits)
drives = createDrives(qubits)

# Create the model
model = Mdl(qubits, coupling + drives)
model.set_lindbladian(False)
model.set_dressed(isDressed)
model.set_FR(useFR)

qubitEnergies = [q.get_Hamiltonian().numpy().diagonal().real / (2 * np.pi) for q in qubits]
qubitEnergies[0] = qubitEnergies[0][::qubit_levels[0]]
qubitEnergies[1] = qubitEnergies[1][:qubit_levels[1]]
qubitTransitions = [np.array([e[i + 1] - e[i] for i in range(len(e) - 1)]) for e in qubitEnergies]
for i in range(len(qubits)):
    print(f"Transmon {i}:")
    print("Energy levels: ", qubitEnergies[i])
    print("Transitions: ", qubitTransitions[i])

qubits frequencies:  [6000000000.0, 5600000000.0] anharmonicities:  [-240000000.0, -220000000.0] coupling:  10000000.0
Transmon 0:
Energy levels:  [0.000e+00 6.000e+09 1.176e+10 1.728e+10 2.256e+10]
Transitions:  [6.00e+09 5.76e+09 5.52e+09 5.28e+09]
Transmon 1:
Energy levels:  [0.000e+00 5.600e+09 1.098e+10 1.614e+10 2.108e+10]
Transitions:  [5.60e+09 5.38e+09 5.16e+09 4.94e+09]


### Load configuration from file or create a new one

In [8]:
def createSpectralDataForRange(limits: Tuple[float, float], num: int, amplitudeFactor=1e-4) -> SpectralData:
    freqs = np.linspace(limits[0], limits[1], num)
    return SpectralData(freqs, np.ones_like(freqs) * amplitudeFactor,
                        np.random.random(len(freqs)) * (2 * np.pi) - np.pi)


def createSpectralDataForRanges(limits: List[Tuple[float, float]], num: List[int], amplitudeFactor=1e-4) -> SpectralData:
    data = SpectralData(np.array([]), np.array([]), np.array([]))
    for i in range(len(limits)):
        freqs = np.linspace(limits[i][0], limits[i][1], num[i])
        data.frequencies = np.append(data.frequencies, np.linspace(limits[i][0], limits[i][1], num[i]))
        data.amplitudes = np.append(data.amplitudes, np.ones_like(freqs) * amplitudeFactor)
        data.phases = np.append(data.phases, np.random.random(len(freqs)) * (2 * np.pi) - np.pi)
    return data


def createSpectralDataFromModel(levelTransitions: List[float], spectralRange: Tuple[float,float]=(3.6e9,5.0e9)) -> SpectralData:
    selectedFrequencies = []
    for t in levelTransitions:
        if spectralRange[0] < t[0] < spectralRange[1]:
            selectedFrequencies.append(t[0])
    selectedFrequencies = np.sort(selectedFrequencies)
    return SpectralData(selectedFrequencies, 0.5e8 * np.ones_like(selectedFrequencies), 0.1 * np.ones_like(selectedFrequencies))


def createSpectralDataFromGeneratedSignal(filename: str, driveName: str) -> SpectralData:
    stored_pmap = PMap()
    stored_pmap.read_config(filename)
    stored_params = stored_pmap.asdict()[list(stored_pmap.asdict().keys())[0]]
    print(stored_params)
    stored_signal = generateSignalFromConfig(stored_params, sim_res=sim_res, awg_res=numPWCPieces / t_final,
                                             useDRAG=False,
                                             usePWC=True, numPWCPieces=60, t_final=t_final)
    return __getFourierBasisFromSignal(stored_signal[driveName]["ts"].numpy(), stored_signal[driveName]["values"].numpy())


def createSpectralDataFromStoredSignal(filename: str) -> SpectralData:
    time, signal = np.load(filename)
    return __getFourierBasisFromSignal(time, signal)


def __getFourierBasisFromSignal(ts, signal) -> SpectralData:
    freq = np.fft.rfftfreq(len(ts), ts[-1] / len(ts))[1:]
    freq_signal = np.fft.rfft(signal)[1:]
    #freq = np.fft.rfftfreq(len(ts), ts[-1] / len(ts))
    #freq_signal = np.abs(np.fft.rfft(values))
    #peaks, amps = findFrequencyPeaks(signal["ts"].numpy(), signal["values"].numpy(), 50, normalise=True)
    return SpectralData(freq, np.abs(freq_signal), np.angle(freq_signal))


def createSpectralData(restrictedRange: Tuple[float, float], frequenciesInRange: int, numPeaks: Tuple[int, int], includedIndices: Tuple[int, int]) -> Dict[str, SpectralData]:
    spectrum: Dict[str, SpectralData] = {}
    for i, drive in enumerate(drives):
        data = createSpectralDataForRange(restrictedRange, frequenciesInRange+2, amplitudeFactor=1e-5 + 7e-3 * (1 - i))
        #data = getFourierBasisFromGeneratedSignal(INPUT_FILE, drive.name)
        #data = SpectralData(
        #    frequenciesISWAP,
        #    np.array([0.015, 0.02, 0.007, 0.005]) if i==0 else np.ones_like(frequenciesISWAP) * 1e-6,
        #    np.ones_like(frequenciesISWAP) * 1e-6
        #)
        print(f"selected frequencies {drive.name}: {len(data.frequencies)}")

        # Restrict to spectral range
        if restrictedRange is not None and len(restrictedRange) > 1 and restrictedRange[0] >= 0 and restrictedRange[1] >= 0:
            data.restrictToRange(restrictedRange[0], restrictedRange[1])
            print(f"Drive {drive.name}: after restriction to range: {len(data.frequencies)}")

        # Select the N largest peaks
        if numPeaks is not None and numPeaks[i] is not None:
            data.selectLargestPeaks(numPeaks[i])
            print(f"Drive {drive.name}: num peaks {len(data.frequencies)}")

        # Select specific frequencies by index
        if includedIndices is not None and includedIndices[i] is not None and len(includedIndices[i]) >= 0:
            data.selectFrequencies(includedIndices[i])
            print(f"Drive {drive.name}: included indices {len(data.frequencies)}")
        spectrum[drive.name] = data
    return spectrum

def getOldConfigurationFromFile(filename: str, spectralRange: Tuple[float, float], numPeaks: Tuple[int, int],
                             includedIndices=(None, None)) -> Dict:
    stored_pmap = PMap()
    stored_pmap.read_config(filename)
    stored_params = stored_pmap.asdict()[list(stored_pmap.asdict().keys())[0]]
    driveChannels = stored_params["drive_channels"]
    for driveIdx, driveName in enumerate(driveChannels.keys()):
        envelope = driveChannels[driveName][f"envelope_{driveName}"]
        data = SpectralData(
            envelope.params["freqs"].get_value().numpy() / (2 * np.pi),
            envelope.params["amps"].get_value().numpy() * envelope.params["amp"].get_value().numpy(),
            envelope.params["phases"].get_value().numpy()
        )

        # restrict to spectral range
        if spectralRange is not None and len(spectralRange) > 1 and spectralRange[0] >= 0 and spectralRange[1] >= 0:
            data.restrictToRange(spectralRange[0], spectralRange[1])
            print(f"Drive {driveName}: after restriction to range: {len(data.frequencies)}")

        # find peaks
        #if numPeaks is not None and numPeaks > 0:
        #    peaks = find_peaks(amps)[0]
        #    freqs = freqs[peaks]
        #    amps = amps[peaks]
        #    phases = phases[peaks]
        #    print(f"Drive {driveName}: peaks found: {len(peaks)}")

        if numPeaks[driveIdx] is not None:
            data.selectLargestPeaks(numPeaks[driveIdx])
            print(f"Drive {driveName}: num peaks {len(data.frequencies)}")

        if includedIndices[driveIdx] is not None:
            data.selectFrequencies(includedIndices[driveIdx])
            print(f"Drive {driveName}: included indices {includedIndices[driveIdx]}")

        envelope.params["freqs"] = Qty(value=data.frequencies, min_val=0.95 * data.frequencies, max_val=1.05 * data.frequencies, unit="Hz 2pi")
        envelope.params["amps"] = Qty(value=data.amplitudes, min_val=1e-2 * data.amplitudes, max_val=1e2 * data.amplitudes, unit="V")
        envelope.params["phases"] = Qty(value=data.phases, min_val=-np.pi * np.ones_like(data.phases),
                                        max_val=np.pi * np.ones_like(data.phases), unit="rad")

        if len(data.frequencies) > 0:
            print(f"frequencies {driveName}: {data.frequencies[0]:e} {data.frequencies[-1]:e} {len(data.frequencies)}")
    return driveChannels

def getNewSpectralDataFromFile(filename: str, spectralRange: Tuple[float, float], numPeaks: Tuple[int, int],
                             includedIndices=(None, None)) -> Dict:
    stored_pmap = PMap()
    stored_pmap.read_config(filename)
    stored_params = stored_pmap.asdict()[list(stored_pmap.asdict().keys())[0]]
    driveChannels = stored_params["drive_channels"]
    spectrum = {}
    for driveIdx, driveName in enumerate(driveChannels.keys()):
        data = SpectralData(np.array([]), np.array([]), np.array([]))

        keys = list(driveChannels[driveName].keys())
        keys = list(filter(lambda x: x.startswith('envelope'), keys))
        for envelopeIdx, envelopeKey in enumerate(keys):
            env = driveChannels[driveName][envelopeKey]
            data.amplitudes = np.append(data.amplitudes, env.params["amp"].get_value())
            data.phases = np.append(data.phases, env.params["xy_angle"].get_value())
            carrier = driveChannels[driveName][f'carrier_{driveName}_{envelopeIdx+1}']
            data.frequencies = np.append(data.frequencies, carrier.params["freq"].get_value() / (2 * np.pi))

        # restrict to spectral range
        if spectralRange is not None and len(spectralRange) > 1 and spectralRange[0] >= 0 and spectralRange[1] >= 0:
            data.restrictToRange(spectralRange[0], spectralRange[1])
            print(f"Drive {driveName}: after restriction to range: {len(data.frequencies)}")

        if numPeaks[driveIdx] is not None:
            data.selectLargestPeaks(numPeaks[driveIdx])
            print(f"Drive {driveName}: num peaks {len(data.frequencies)}")

        if includedIndices[driveIdx] is not None:
            data.selectFrequencies(includedIndices[driveIdx])
            print(f"Drive {driveName}: included indices {includedIndices[driveIdx]}")

        if len(data.frequencies) > 0:
            print(f"frequencies {driveName}: {data.frequencies[0]:e} {data.frequencies[-1]:e} {len(data.frequencies)}")
        spectrum[driveName] = data
    return spectrum

def getNewConfigurationFromFile(filename: str, spectralRange: Tuple[float, float], numPeaks: Tuple[int, int], includedIndices: Tuple[List[int], List[int]]=(None, None)) -> Dict:
    stored_pmap = PMap()
    stored_pmap.read_config(filename)
    stored_params = stored_pmap.asdict()[list(stored_pmap.asdict().keys())[0]]
    driveChannels = stored_params["drive_channels"]

    for driveIdx, driveName in enumerate(driveChannels.keys()):
        keys = list(filter(lambda x: x.startswith('envelope'), list(driveChannels[driveName].keys())))
        amplitudeList = np.zeros(len(keys))

        for envelopeIdx, envelopeKey in enumerate(keys):
            env = driveChannels[driveName][envelopeKey]
            carrierKey = f'carrier_{driveName}_{envelopeIdx+1}'
            carrier = driveChannels[driveName][carrierKey]

            # restrict to spectral range
            if spectralRange is not None and len(spectralRange) > 1 and spectralRange[0] >= 0 and spectralRange[1] >= 0:
                freq = carrier.params["freq"].get_value() / (2 * np.pi)
                if freq < spectralRange[0] or freq > spectralRange[1]:
                    del driveChannels[driveName][envelopeKey]
                    del driveChannels[driveName][carrierKey]

            # extract all amplitudes
            if numPeaks[driveIdx] is not None:
                if "ampR" in env.params and "ampI" in env.params:
                    amplitudeList[envelopeIdx] = np.sqrt(
                        env.params["ampR"].get_value() ** 2 + env.params["ampI"].get_value() ** 2)
                elif "amp" in env.params:
                    amplitudeList[envelopeIdx] = env.params["amp"].get_value()
                else:
                    raise Exception("Neither amp nor ampR and ampI found in envelope parameters")

        # filter out amplitudes
        if numPeaks[driveIdx] is not None:
            indices = amplitudeList.argsort()
            indicesRemove = indices[0:len(keys) - numPeaks[driveIdx]]
            for idx in indicesRemove:
                del driveChannels[driveName][f'envelope_{driveName}_{idx+1}']
                del driveChannels[driveName][f'carrier_{driveName}_{idx+1}']

        # select only specific frequencies
        if includedIndices[driveIdx] is not None:
            removeIndices = [x for x in range(len(keys)) if x not in includedIndices[driveIdx]]
            for idx in removeIndices:
                del driveChannels[driveName][f'envelope_{driveName}_{idx + 1}']
                del driveChannels[driveName][f'carrier_{driveName}_{idx + 1}']

        envelopes = list(filter(lambda x: x.startswith('envelope'), list(driveChannels[driveName].keys())))
        carriers = list(filter(lambda x: x.startswith('carrier'), list(driveChannels[driveName].keys())))
        print(f"envelopes {driveName}: {len(envelopes)}")
        print(f"carriers {driveName}: {len(carriers)}")
    return driveChannels

def convertOldConfigurationToSpectrum(configuration: Dict) -> Dict[str, SpectralData]:
    spectrum = {}
    for drive in ["d1", "d2"]:
        params = configuration[drive][f"envelope_{drive}"].params
        spectrum[drive] = SpectralData(params["freqs"], params["amps"], params["phases"])
    return spectrum

def convertNewConfigurationToSpectrum(configuration: Dict) -> Dict[str, SpectralData]:
    spectrum = {}
    for driveName in ["d1", "d2"]:
        keys = list(filter(lambda x: x.startswith('envelope'), list(configuration[driveName].keys())))
        data = SpectralData(np.array([]), np.array([]), np.array([]))
        for idx, key in enumerate(keys):
            env = configuration[driveName][key]
            data.amplitudes = np.append(data.amplitudes, env.params["amp"].get_value())
            data.phases = np.append(data.phases, env.params["xy_angle"].get_value())
            carrier = configuration[driveName][f'carrier_{key[9:]}']
            data.frequencies = np.append(data.frequencies, carrier.params["freq"].get_value() / (2 * np.pi))
        spectrum[driveName] = data
    return spectrum

stored_params_old = None
stored_params_new = None

spectrum = createSpectralData(SPECTRAL_RANGE, FREQUENCIES_IN_RANGE, NUM_PEAKS, INCLUDED_INDICES)
#spectrum = getNewSpectralDataFromFile(INPUT_FILE, spectralRange=SPECTRAL_RANGE, numPeaks=NUM_PEAKS, includedIndices=INCLUDED_INDICES)
#spectrum = {
#    "d1": createSpectralDataForRanges([(3.9e9, 4.1e9), (4.4e9, 4.75e9)], [100, 200], amplitudeFactor=1e-3),
#    "d2": createSpectralDataForRanges([(3.9e9, 4.1e9), (4.4e9, 4.75e9)], [100, 200], amplitudeFactor=1e-5)
#}

#stored_params_old = getOldConfigurationFromFile(INPUT_FILE, spectralRange=SPECTRAL_RANGE, numPeaks=NUM_PEAKS, includedIndices=INCLUDED_INDICES)
#spectrum = convertOldConfigurationToSpectrum(stored_params_old)
#stored_params_old = None
#stored_params_new = getNewConfigurationFromFile(INPUT_FILE, spectralRange=SPECTRAL_RANGE, numPeaks=NUM_PEAKS, includedIndices=INCLUDED_INDICES)
#spectrum = convertNewConfigurationToSpectrum(stored_params_new)

for d in ["d1", "d2"]:
    print(f"{d}: {spectrum[d]}")
    print(spectrum[d].frequencies)
    print(spectrum[d].amplitudes)

selected frequencies d1: 202
Drive d1: after restriction to range: 200
Drive d1: num peaks 200
selected frequencies d2: 202
Drive d2: after restriction to range: 200
Drive d2: num peaks 200
d1: 200 frequencies
[6.29104478e+09 5.06417910e+09 5.15373134e+09 5.14477612e+09 5.13582090e+09 5.12686567e+09 5.11791045e+09 5.10895522e+09 5.10000000e+09 5.09104478e+09 5.08208955e+09 5.07313433e+09 5.05522388e+09 6.28208955e+09 5.04626866e+09 5.03731343e+09 5.02835821e+09 5.01940299e+09 5.01044776e+09
 5.00149254e+09 4.99253731e+09 4.98358209e+09 4.97462687e+09 4.96567164e+09 5.16268657e+09 5.17164179e+09 5.18059701e+09 5.18955224e+09 5.37761194e+09 5.36865672e+09 5.35970149e+09 5.35074627e+09 5.34179104e+09 5.33283582e+09 5.32388060e+09 5.31492537e+09 5.30597015e+09 5.29701493e+09
 5.28805970e+09 5.27910448e+09 5.27014925e+09 5.26119403e+09 5.25223881e+09 5.24328358e+09 5.23432836e+09 5.22537313e+09 5.21641791e+09 5.20746269e+09 5.19850746e+09 4.95671642e+09 4.94776119e+09 4.93880597e+09 4.71492

### Create the generator with all devices

In [9]:
# Envelopes and carriers
#carrier_freqs = [1e-8, 1e-8]
#carrier_freqs = [1e-5, 1e-5]
#carrier_framechange = [0.00, 0.00]
#pulse_t_final = [t_final, t_final]
#pulse_sigmas = [t_final/5, t_final/5]
#pulse_amps = [0.5, 0.5]
#pulse_deltas = [0, 0]
#pulse_xy_angles = [0.0, 0.0]
#pulse_freq_offsets = [0.0, 0.0]

print("Generator: N=", len(spectrum[drives[0].name].frequencies))
generator = createGeneratorNLOs(drives, sim_res=sim_res, awg_res=awg_res, N=len(spectrum[drives[0].name].frequencies))

envelopes = []
envelopesForDrive = {d.name: [] for d in drives}
carriers = []
carriersForDrive = {d.name: [] for d in drives}

#selectedFreqsMap, selectedAmpsMap, selectedPhasesMap = getFourierBasisFromModel(transitions)
#stored_params = getConfigurationFromFile("./optimised_params/10freqsvariable_from15_CZ.json", spectralRange=(3.5e9, 5.5e9), numPeaks=10)
#selectedFreqsMap, selectedAmpsMap, selectedPhasesMap = getFourierBasisFromSignal(INPUT_FILE)
#selectedFreqs = np.array(np.abs(directTransitions))
#selectedAmps = np.ones_like(directTransitions) * 0.1
#selectedFreqs = np.array([1e9])
#selectedAmps = 0.1 * np.ones_like(selectedFreqs)
#selectedPhases = np.ones_like(selectedFreqs)
#print(selectedFreqs)

if stored_params_old is not None:
    print("Creating envelopes from configuration")
    for idx in [0, 1]:
        driveName = drives[idx].name
        stored_params_drive = stored_params_old[driveName]
        # Copy the envelope
        env: pulse.Envelope = copy.deepcopy(stored_params_drive[f"envelope_{driveName}"])
        if useDRAG and not isinstance(env, EnvelopeDrag):
            print("Converting to DRAG")
            env = convertToDRAG(env)
        elif not useDRAG and isinstance(env, EnvelopeDrag):
            print("Converting from DRAG")
            env = convertFromDRAG(env)
        env.params["t_final"] = Qty(value=t_final, min_val=0.5*t_final, max_val=1.5*t_final, unit="s")
        envelopes.append(env)
        envelopesForDrive[driveName].append(env)

        # Copy the carrier
        carrier = copy.deepcopy(stored_params_drive[f"carrier_{driveName}"])
        carrier.name = f"carrier_{driveName}"
        carriers.append(carrier)
        carriersForDrive[driveName].append(carrier)
if stored_params_new is not None:
    print("Creating envelopes from configuration")
    for idx in [0, 1]:
        driveName = drives[idx].name
        stored_params_drive = stored_params_new[driveName]

        envelopeKeys = list(filter(lambda x: x.startswith('envelope'), list(stored_params_drive.keys())))
        for i, key in enumerate(envelopeKeys):
            # Copy the envelope
            env: pulse.Envelope = copy.deepcopy(stored_params_drive[key])
            env.name = f"envelope_{driveName}_{i + 1}"
            if useDRAG and not isinstance(env, EnvelopeDrag):
                print("Converting to DRAG")
                env = convertToDRAG(env)
            elif not useDRAG and isinstance(env, EnvelopeDrag):
                print("Converting from DRAG")
                env = convertFromDRAG(env)
            env.params["t_final"] = Qty(value=t_final, min_val=0.5 * t_final, max_val=1.5 * t_final, unit="s")
            #env.params["amp"] = Qty(value=env.params["amp"].get_value(),
            #                        min_val=0.1 * env.params["amp"].get_value(),
            #                        max_val=10.0 * env.params["amp"].get_value())
            env.params["ampR"] = Qty(value=env.params["ampR"].get_value(),
                                     min_val=0.1 * env.params["ampR"].get_value(),
                                     max_val=10.0 * env.params["ampR"].get_value())
            env.params["ampI"] = Qty(value=env.params["ampI"].get_value(),
                                     min_val=0.1 * env.params["ampI"].get_value(),
                                     max_val=10.0 * env.params["ampI"].get_value())
            envelopes.append(env)
            envelopesForDrive[driveName].append(env)

            # Copy the carrier
            carrierKey = f"carrier_{key[9:]}"
            carrier = copy.deepcopy(stored_params_drive[carrierKey])
            carrier.name = f"carrier_{driveName}_{i+1}"
            carriers.append(carrier)
            carriersForDrive[driveName].append(carrier)
elif spectrum is not None:
    print("Creating envelopes from spectral data")
    for idx in [0, 1]:
        data = spectrum[drives[idx].name]
        for k in range(len(data.frequencies)):
            # Create an envelope
            if USE_COMPLEX_GAUSSIANS:
                env = createComplexGaussianPulse(
                t_final=t_final,
                sigmaR=t_final * 10,
                sigmaI=t_final * 10,
                amp=1.0 if USE_COMPLEX_AMPLITUDE else (3.0/4.4)*data.amplitudes[k],
                ampR=(3.0/4.4)*data.amplitudes[k] if USE_COMPLEX_AMPLITUDE else None,
                ampI=(3.0/4.4)*data.amplitudes[k] if USE_COMPLEX_AMPLITUDE else None,
                delta=0.0,
                xy_angle=data.phases[k],
                freq_off=1e-5,
                useDrag=useDRAG
                )
            else:
                env = createGaussianPulse(
                    t_final=t_final,
                    sigma=t_final * 10,
                    amp=data.amplitudes[k],
                    delta=0.0,
                    xy_angle=data.phases[k],
                    freq_off=1e-5,
                    useDrag=useDRAG
                )
            if usePWC:
                env = convertToPWC(env, numPWCPieces)
            env.name = f"envelope_{drives[idx].name}_{k+1}"
            envelopes.append(env)
            envelopesForDrive[drives[idx].name].append(env)

            # Create a carrier
            carrier_parameters = {
                "freq": Qty(value=data.frequencies[k], min_val=0.98 * data.frequencies[k],
                            max_val=1.02 * data.frequencies[k], unit="Hz 2pi"),
                "framechange": Qty(value=0.0, min_val=-np.pi, max_val=3 * np.pi, unit="rad"),
            }
            carrier = pulse.Carrier(
                name=f"carrier_{drives[idx].name}_{k+1}",
                desc="Frequency of the local oscillator",
                params=carrier_parameters,
            )
            carriers.append(carrier)
            carriersForDrive[drives[idx].name].append(carrier)

print("frequencies: ", [[carrier.params["freq"] for carrier in carriers] for carriers in carriersForDrive.values()])
print("amp: ", [[env.params["amp"] for env in envelopes] for envelopes in envelopesForDrive.values()])
print("phases: ", [[env.params["xy_angle"] for env in envelopes] for envelopes in envelopesForDrive.values()])

Generator: N= 200
Creating envelopes from spectral data
frequencies:  [[6.291 GHz 2pi, 5.064 GHz 2pi, 5.154 GHz 2pi, 5.145 GHz 2pi, 5.136 GHz 2pi, 5.127 GHz 2pi, 5.118 GHz 2pi, 5.109 GHz 2pi, 5.100 GHz 2pi, 5.091 GHz 2pi, 5.082 GHz 2pi, 5.073 GHz 2pi, 5.055 GHz 2pi, 6.282 GHz 2pi, 5.046 GHz 2pi, 5.037 GHz 2pi, 5.028 GHz 2pi, 5.019 GHz 2pi, 5.010 GHz 2pi, 5.001 GHz 2pi, 4.993 GHz 2pi, 4.984 GHz 2pi, 4.975 GHz 2pi, 4.966 GHz 2pi, 5.163 GHz 2pi, 5.172 GHz 2pi, 5.181 GHz 2pi, 5.190 GHz 2pi, 5.378 GHz 2pi, 5.369 GHz 2pi, 5.360 GHz 2pi, 5.351 GHz 2pi, 5.342 GHz 2pi, 5.333 GHz 2pi, 5.324 GHz 2pi, 5.315 GHz 2pi, 5.306 GHz 2pi, 5.297 GHz 2pi, 5.288 GHz 2pi, 5.279 GHz 2pi, 5.270 GHz 2pi, 5.261 GHz 2pi, 5.252 GHz 2pi, 5.243 GHz 2pi, 5.234 GHz 2pi, 5.225 GHz 2pi, 5.216 GHz 2pi, 5.207 GHz 2pi, 5.199 GHz 2pi, 4.957 GHz 2pi, 4.948 GHz 2pi, 4.939 GHz 2pi, 4.715 GHz 2pi, 4.697 GHz 2pi, 4.688 GHz 2pi, 4.679 GHz 2pi, 4.670 GHz 2pi, 4.661 GHz 2pi, 4.652 GHz 2pi, 4.643 GHz 2pi, 4.634 GHz 2pi, 4.625 GHz 2pi

### Ideal gate

In [ ]:
printMatrix(IDEAL_GATE, createTwoTransmonsQubitLabels(includeLeakage=False, makeLatexKets=True), "ideal_gate", output)

gate = gates.Instruction(
    name=IDEAL_GATE_NAME,
    #name="unity",
    targets=[0, 1],
    t_start=0.0,
    t_end=t_final,
    channels=[d.name for d in drives],
    ideal=IDEAL_GATE,
)
for drive in drives:
    for env in envelopesForDrive[drive.name]:
        gate.add_component(copy.deepcopy(env), drive.name)
    for carrier in carriersForDrive[drive.name]:
        gate.add_component(copy.deepcopy(carrier), drive.name)

### Set up the experiment and compute propagators

In [ ]:
# Set up the experiment
parameter_map = PMap(instructions=[gate], model=model, generator=generator)
exp = Exp(pmap=parameter_map, sim_res=sim_res)
exp.set_opt_gates([gate.get_key()])

unitaries = exp.compute_propagators()
printPropagator(exp, gate, createTwoTransmonsQubitLabels(includeLeakage=True, makeLatexKets=True), createTwoTransmonsQubitLabels(includeLeakage=False, makeLatexKets=True), output)
#printAllSignals(exp, qubits[0], output, directory="devices_before")
#printAllSignals(exp, qubits[1], output, directory="devices_before")

In [ ]:
initialFidelity = 1 - tf_utils.tf_unitary_overlap(tf_project_to_comp(
        unitaries[gate.get_key()], dims=[25], index=[0], outdims=[16]
    ), IDEAL_GATE, lvls=4).numpy()
print("Initial infidelity: ", initialFidelity)

### Calculate and save the time-dependent Hamiltonian

In [ ]:
'''
signal = parameter_map.generator.generate_signals(parameter_map.instructions[gate.get_key()])
H = parameter_map.model.get_Hamiltonian(signal)
print(H.shape)
output.save(H, 'hamiltonian_timedependent')
'''

### Print the generated signal

In [ ]:
# Obtain all energies and transitions
#stateEnergies = getEnergiesFromFile('./optimised_params/fourier basis 5 levels/level_energies.npy', level_labels_short)
#stateEnergies = getEnergiesFromModel(exp, gate, level_labels_short)
stateEnergies = getEnergiesFromPropagator(unitaries[gate.get_key()], t_final)
transitions = calculateTransitions(stateEnergies, createTwoTransmonsStateLabels())
#for t in transitions:
#    print(t)

#directTransitions = np.array([np.abs(stateEnergies[i + 1][0] - stateEnergies[i][0]) for i in range(0, len(stateEnergies) - 1, 1)])
#directTransitions = []
#for i in range(0, len(stateEnergies)-1, 1):
#    diff = stateEnergies[i+1][0] - stateEnergies[i][0]
#    directTransitions.append(diff)
#print(directTransitions)

printSignal(exp, qubits, gate, output=output, states=transitions)
#printAllSignals(exp, qubits[0], output, directory="devices_before")
parameter_map.write_config(output.createFileName("parameter_map", "json"))

In [ ]:
# Find all energy eigenvalues of the full Hamiltonian calculated from the propagator over short times
'''
U = unitaries[gate.get_key()].numpy()
interval = 1e-12
tau = t_final / interval
U = np.float_power(U, 1.0 / tau)
diag = np.diagonal(U)
#print(diag)
energies = -np.angle(diag) / (2 * np.pi * interval)
print(energies)
'''
'''
# all energy levels with labels
stateEnergies = zip(energies, level_labels_short)

# all energy transitions
items = sorted(stateEnergies, key=lambda x: x[0])
transitions = []
for i in range(len(items)):
    for j in range(len(items)):
        if i != j:
            E = items[j][0] - items[i][0]
            if E > 0:
                transitions.append((E, items[i][1] + " - " + items[j][1]))

transitions.sort(key=lambda x: x[0])
#for t in transitions:
#    print(t)

#directTransitions = np.array([np.abs(stateEnergies[i + 1][0] - stateEnergies[i][0]) for i in range(0, len(stateEnergies) - 1, 1)])
#directTransitions = []
#for i in range(0, len(stateEnergies)-1, 1):
#    diff = stateEnergies[i+1][0] - stateEnergies[i][0]
#    directTransitions.append(diff)
#print(directTransitions)
'''

'''
# Read the Stark-shifted energies from a file
energies = np.load('/home/user/eigenvalues.npy')

# Combine energies with state labels
stateEnergies = zip(energies, level_labels_short)
for energy,state in zip(energies, level_labels_short):
    print(state, energy)

# Compute all resonance energies
items = sorted(stateEnergies, key=lambda x: x[0])
transitions = []
for i in range(len(items)):
    for j in range(len(items)):
        if i != j:
            E = items[j][0] - items[i][0]
            if E > 0:
                transitions.append((E, items[i][1] + " - " + items[j][1]))

transitions.sort(key=lambda x: x[0])
for t in transitions:
    print(t)
'''

In [ ]:
# Plot time dependent populations of the basis states for different initial states
'''
filenameLabels = createTwoTransmonsStateLabels(includeLeakage=True, labelLeakageAsNumbers=True, makeLatexKets=False)
plotLabels = createTwoTransmonsStateLabels(includeLeakage=True, labelLeakageAsNumbers=True, makeLatexKets=True)
for i in [0]: #range(model.tot_dim):
    print(i)
    psi_init = [[0] * model.tot_dim]
    psi_init[0][i] = 1
    psi_init /= np.linalg.norm(psi_init)
    print("initial state: ", psi_init)
    init_state = tf.transpose(tf.constant(psi_init, tf.complex128))
    sequence = [gate.get_key()]

    populations = calculatePopulation(exp, init_state, [gate.get_key()])
    output.save(populations, f"population_{filenameLabels[i]}")
    plotPopulation(exp, populations, sequence=[gate.get_key()],
                   labels=plotLabels, filename=output.createFileName(f"population_{filenameLabels[i]}", "svg"))
    #phases = calculatePhases(exp, init_state, [gate.get_key()], stepFraction=1)
    #phases = phases[0,:].reshape((1, phases.shape[1]))
    #output.save(phases, f"phases_{filenameLabels[i]}")
    #plotPopulation(exp, phases, sequence=[gate.get_key()],
    #               labels=plotLabels, filename=output.createFileName(f"phases_{filenameLabels[i]}", "svg"))
'''
#printEntanglementEvolution(exp, gate, output)

In [ ]:
# Specify the parameters to be optimised and initialise the optimiser
opt_map = []
for drive in [drives[i] for i in OPTIMISABLE_DRIVES]:
    for env in envelopesForDrive[drive.name]:
        if USE_COMPLEX_AMPLITUDE:
            opt_map.append([(gate.get_key(), drive.name, env.name, "ampR")])
            opt_map.append([(gate.get_key(), drive.name, env.name, "ampI")])
        else:
            opt_map.append([(gate.get_key(), drive.name, env.name, "amp")])
        #opt_map.append([(gate.get_key(), drive.name, env.name, "freq_offset")])
        opt_map.append([(gate.get_key(), drive.name, env.name, "xy_angle")])
        opt_map.append([(gate.get_key(), drive.name, env.name, "sigmaR")])
        opt_map.append([(gate.get_key(), drive.name, env.name, "sigmaI")])
        if useDRAG:
            opt_map.append([(gate.get_key(), drive.name, env.name, "delta")])
        if usePWC:
            opt_map.append([(gate.get_key(), drive.name, env.name, "inphase")])
            opt_map.append([(gate.get_key(), drive.name, env.name, "quadrature")])
            #opt_map.append([(gate.get_key(), drive.name, env.name, "t_bin_end")])
        #else:
        #    opt_map.append([(gate.get_key(), drive.name, env.name, "amps")])
        #    if OPTIMISE_FREQUENCIES:
        #        opt_map.append([(gate.get_key(), drive.name, env.name, "freqs")])
        #    opt_map.append([(gate.get_key(), drive.name, env.name, "phases")])
        #    if OPTIMISE_TFINAL:
        #        opt_map.append([(gate.get_key(), drive.name, env.name, "t_final")])
    for carrier in carriersForDrive[drive.name]:
        opt_map.append([(gate.get_key(), drive.name, carrier.name, "freq")])
    #    #opt_map.append([(gate.get_key(), drive.name, carrier.name, "framechange")])
parameter_map.set_opt_map(opt_map)
parameter_map.print_parameters()

In [ ]:
infidelities = []

for algorithm, params in selected_algorithms:
    if algorithm == ALGORITHM_LBFGS:
        infidelities = optimise(output, qubits, exp, algorithms.lbfgs, {
            "maxfun": 50,
            "ftol": 1e-5,
            **params
        }, gate)
    elif algorithm == ALGORITHM_LBFGS_GRAD_FREE:
        infidelities = optimise(output, qubits, exp, algorithms.lbfgs_grad_free, {
            "maxfun": 1000,
            "gtol": 1e-4,
            "ftol": 1e-4,
            **params
        }, gate)
    elif algorithm == ALGORITHM_CMAES:
        infidelities = optimise(output, qubits, exp, algorithms.cmaes, {
            "popsize": 15,
            "spread": 0.02,
            "maxfevals": 2000,
            "init_point": "True",
            "stop_at_sigma": 1e-3,
            "stop_at_convergence": 20,
            **params
        }, gate)
    elif algorithm == ALGORITHM_GCMAES:
        infidelities = optimise(output, qubits, exp, algorithms.gcmaes, {
            "cmaes": {"popsize": 12, "spread": 0.05, "maxfevals": 20,
                      "init_point": "True", "stop_at_sigma": 1e-4, "stop_at_convergence": 20, **params["cmaes"]},
            "lbfgs": {"maxfun": 500, "ftol": 1e-6, **params["lbfgs"]}
        }, gate)
    else:
        print("Unknown algorithm: ", algorithm)

In [ ]:
# Plot results
output = DataOutput(output_dir, file_suffix='after')
plotData(np.arange(len(infidelities)), infidelities, xlabel="Step",
         ylabel="Infidelity", filename=output.createFileName("convergence", "svg"))
printSignal(exp, qubits, gate, output=output, states=transitions)
#printAllSignals(exp, qubits, output, directory="devices_after")
printPropagator(exp, gate, createTwoTransmonsQubitLabels(includeLeakage=True, makeLatexKets=True), createTwoTransmonsQubitLabels(includeLeakage=False, makeLatexKets=True), output, savePartials=False)
#printTimeEvolution(exp, init_state, gate, level_labels, output)
#printEntanglementEvolution(exp, gate, output)
parameter_map.write_config(output.createFileName("parameter_map", "json"))